In [1]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "38002279fa574fbcba0fbc424ae056d5"
os.environ["OPENAI_API_BASE"] = "https://lumin-azure-openai-rnd.openai.azure.com/"

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    #api_type="azure",
    api_key=os.environ.get("OPENAI_API_KEY"),
    #api_base=os.getenv("OPENAI_API_BASE"),
    #api_version="2023-03-15-preview",
)

NameError: name 'openai' is not defined

In [6]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="lumin-rnd-gpt-4-32k",
)
chat_completion

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 38002279********************56d5. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [2]:
!pip install openai==1.2.0 --upgrade

  Using cached https://files.pythonhosted.org/packages/42/9d/b227c396c6eafbece9a8fe877379582481b108c0c0cf4ff8770ce63926ad/openai-1.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2a/14/e75e52d521442e2fcc9f1df3c5e456aead034203d4797867980de558ab34/tqdm-4.66.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.27.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e5/f3/8296f550276194a58